In [1]:
import sys
import importlib.util

file_path = '/kaggle/input/classok/classok.py'
module_name = 'classok'

spec = importlib.util.spec_from_file_location(module_name, file_path)
classok = importlib.util.module_from_spec(spec)
sys.modules[module_name] = classok
spec.loader.exec_module(classok)

2025-06-20 20:25:57.314016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750451157.542096      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750451157.606485      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from classok import ImageCaptioner

# IdeficsForVisionText2Text modellt használva

In [4]:
from transformers import AutoProcessor, IdeficsForVisionText2Text
import torch
import json

model_id = "HuggingFaceM4/idefics-9b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

processor = AutoProcessor.from_pretrained(model_id)
model = IdeficsForVisionText2Text.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto",
    offload_buffers=True
)

# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (
"""User: Provide a detailed description of the flower in the image:

    Example: "The poinsettia has large, vividly colored bracts, often mistaken for petals. The bracts form a star-shaped rosette around small yellow flowers in the center. The bracts come in shades of red, white, pink, orange, or variegated blends, creating a strong contrast with dark green leaves. Leaf texture is smooth or lightly veined, with pointed or rounded tips depending on the cultivar. The flowers are yellow-green and surrounded by tiny red glands that secrete nectar."

    1. What is the overall shape of the flower's bloom? Is it symmetrical or asymmetrical? Describe the petals' shape.
    2. What color are the petals? Are there any gradients or patterns? How does the color change in different light?
    3. Describe the texture of the petals. Are they smooth, velvety, or waxy? Do they reflect light, appearing glossy or matte?
    4. What does the center of the flower look like? Are there visible stamens or pistils? What colors and textures are visible?
    5. What is the size of the flower's bloom? Is it compact or spread out?
    6. How are the petals arranged? Are they spaced evenly, overlapping, or layered? Are there any visible veins or markings?
    7. Are leaves visible? Describe their shape, color, and texture.
    8. What is the condition of the flower? Is it fully open, partially open, or wilting?
    9. Is the flower growing alone or in a group? How are the flowers arranged? Are they at different blooming stages?
    10. Where is the flower found? Is it in a garden, field, or forest? Describe the surroundings.
    11. How does the flower appear under different lighting? Does it have shadows, translucence, or reflections?
    12. Is the flower in full bloom or at a particular stage? How does it compare to other flowers in different blooming stages?
    13. What visual features distinguish this flower, such as shape, color, or texture?
    14. How does the environment affect the flower’s appearance? Is it in rocky, moist, or dry soil?

    Assistant:

    """
)

captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/idefics_flower_captions.json",
    prompt=prompt_template,
    model_type="idefics",
    device=device,
    dtype=dtype
)


custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Captioning: /kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg


# Blip2 modelt használva

In [3]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
import json

model_id = "Salesforce/blip2-flan-t5-xl"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16  # works best with BLIP-2 on GPU

# Load processor and model for BLIP-2
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto"
)


# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (""" 1. What is the overall shape of the flower's bloom? Is it symmetrical or asymmetrical? Describe the petals' shape.
    2. What color are the petals? Are there any gradients or patterns? How does the color change in different light?
    3. Describe the texture of the petals. Are they smooth, velvety, or waxy? Do they reflect light, appearing glossy or matte?
    4. What does the center of the flower look like? Are there visible stamens or pistils? What colors and textures are visible?
    5. What is the size of the flower's bloom? Is it compact or spread out?
    6. How are the petals arranged? Are they spaced evenly, overlapping, or layered? Are there any visible veins or markings?
    7. Are leaves visible? Describe their shape, color, and texture.
    8. What is the condition of the flower? Is it fully open, partially open, or wilting?
    9. Is the flower growing alone or in a group? How are the flowers arranged? Are they at different blooming stages?
    10. Where is the flower found? Is it in a garden, field, or forest? Describe the surroundings.
    11. How does the flower appear under different lighting? Does it have shadows, translucence, or reflections?
    12. Is the flower in full bloom or at a particular stage? How does it compare to other flowers in different blooming stages?
    13. What visual features distinguish this flower, such as shape, color, or texture?
    14. How does the environment affect the flower’s appearance? Is it in rocky, moist, or dry soil?"""
)




captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/blip2_flower_captions.json",
    prompt=prompt_template,
    model_type="blip2",
    device=device,
    dtype=dtype
)


custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


Captioning: /kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


# Llava modelt használva

In [3]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration
import torch
import json

model_id = "llava-hf/llava-1.5-7b-hf"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

processor = LlavaProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto"
)




# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (
    """<image>\nUSER:  1. What is the overall shape of the flower's bloom? Is it symmetrical or asymmetrical? Describe the petals' shape.
    2. What color are the petals? Are there any gradients or patterns? How does the color change in different light?
    3. Describe the texture of the petals. Are they smooth, velvety, or waxy? Do they reflect light, appearing glossy or matte?
    4. What does the center of the flower look like? Are there visible stamens or pistils? What colors and textures are visible?
    5. What is the size of the flower's bloom? Is it compact or spread out?
    6. How are the petals arranged? Are they spaced evenly, overlapping, or layered? Are there any visible veins or markings?
    7. Are leaves visible? Describe their shape, color, and texture.
    8. What is the condition of the flower? Is it fully open, partially open, or wilting?
    9. Is the flower growing alone or in a group? How are the flowers arranged? Are they at different blooming stages?
    10. Where is the flower found? Is it in a garden, field, or forest? Describe the surroundings.
    11. How does the flower appear under different lighting? Does it have shadows, translucence, or reflections?
    12. Is the flower in full bloom or at a particular stage? How does it compare to other flowers in different blooming stages?
    13. What visual features distinguish this flower, such as shape, color, or texture?
    14. How does the environment affect the flower’s appearance? Is it in rocky, moist, or dry soil?\nASSISTANT:"""
)



captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/llava_flower_captions.json",
    prompt=prompt_template,
    model_type="llava",
    device=device,
    dtype=dtype
)




custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]


Captioning: /kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg
